In [20]:
# modules we'll use
import pandas as pd
import numpy as np
import sklearn

# set seed for reproducibility
np.random.seed(0) 

In [ ]:
# read in all our data
nfl_data = pd.read_csv("NFL Play by Play 2009-2017 (v4).csv")
sf_permits = pd.read_csv("Building_Permits.csv")

# Check missing value stats

In [7]:
#nfl_data.head(3)
def missing_data(data):
    total = data.isnull().sum()
    percent = (data.isnull().sum()/data.isnull().count()*100)
    tt = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
    types = []
    for col in data.columns:
        dtype = str(data[col].dtype)
        types.append(dtype)
    tt['Types'] = types
    tt=tt.sort_values(by=['Percent'],ascending=False)
    #return(np.transpose(tt))
    return tt

missing_data(nfl_data)

,Total,Percent,Types
DefTwoPoint,407664,99.994113,object
BlockingPlayer,407571,99.971302,object
TwoPointConv,407083,99.851602,object
ChalReplayResult,404286,99.165538,object
RecFumbTeam,403315,98.927366,object
RecFumbPlayer,403315,98.927366,object
Interceptor,403168,98.891309,object
FieldGoalDistance,398740,97.805184,float64
FieldGoalResult,398629,97.777958,object
ExPointResult,397578,97.520162,object


In [5]:
# how many total missing values do we have?
total_cells = np.product(nfl_data.shape)
total_missing = missing_values_count.sum()

# percent of data that is missing
(total_missing/total_cells) * 100

24.87214126835169

In [8]:
# remove all the rows that contain a missing value
rows_with_na_dropped=nfl_data.dropna(axis=0)
rows_with_na_dropped.head()

# just how much data did we lose?
print("Rows in original dataset: %d \n" % nfl_data.shape[0])
print("Rows with na's dropped: %d" % rows_with_na_dropped.shape[0])

Rows in original dataset: 407688 

Rows with na's dropped: 0


In [9]:
# remove all columns with at least one missing value
columns_with_na_dropped = nfl_data.dropna(axis=1)
columns_with_na_dropped.head()

# just how much data did we lose?
print("Columns in original dataset: %d \n" % nfl_data.shape[1])
print("Columns with na's dropped: %d" % columns_with_na_dropped.shape[1])

Columns in original dataset: 102 

Columns with na's dropped: 41


# Handling missing values 

In [10]:
#Method 1 

# get a small subset of the NFL dataset
subset_nfl_data = nfl_data.loc[:, 'EPA':'Season'].head()
subset_nfl_data

# replace all NA's with 0
subset_nfl_data.fillna(0)

,EPA,airEPA,yacEPA,Home_WP_pre,Away_WP_pre,Home_WP_post,Away_WP_post,Win_Prob,WPA,airWPA,yacWPA,Season
0,2.014474,0.000000,0.000000,0.485675,0.514325,0.546433,0.453567,0.485675,0.060758,0.000000,0.000000,2009
1,0.077907,-1.068169,1.146076,0.546433,0.453567,0.551088,0.448912,0.546433,0.004655,-0.032244,0.036899,2009
2,-1.402760,0.000000,0.000000,0.551088,0.448912,0.510793,0.489207,0.551088,-0.040295,0.000000,0.000000,2009
3,-1.712583,3.318841,-5.031425,0.510793,0.489207,0.461217,0.538783,0.510793,-0.049576,0.106663,-0.156239,2009
4,2.097796,0.000000,0.000000,0.461217,0.538783,0.558929,0.441071,0.461217,0.097712,0.000000,0.000000,2009


In [11]:
#Method 2 

# replace all NA's the value that comes directly after it in the same column, 
# then replace all the reamining na's with 0
subset_nfl_data.fillna(method = 'bfill', axis=0).fillna(0)

,EPA,airEPA,yacEPA,Home_WP_pre,Away_WP_pre,Home_WP_post,Away_WP_post,Win_Prob,WPA,airWPA,yacWPA,Season
0,2.014474,-1.068169,1.146076,0.485675,0.514325,0.546433,0.453567,0.485675,0.060758,-0.032244,0.036899,2009
1,0.077907,-1.068169,1.146076,0.546433,0.453567,0.551088,0.448912,0.546433,0.004655,-0.032244,0.036899,2009
2,-1.402760,3.318841,-5.031425,0.551088,0.448912,0.510793,0.489207,0.551088,-0.040295,0.106663,-0.156239,2009
3,-1.712583,3.318841,-5.031425,0.510793,0.489207,0.461217,0.538783,0.510793,-0.049576,0.106663,-0.156239,2009
4,2.097796,0.000000,0.000000,0.461217,0.538783,0.558929,0.441071,0.461217,0.097712,0.000000,0.000000,2009


In [18]:
#Method 3 
from sklearn.impute import SimpleImputer
my_imputer = SimpleImputer()
print(nfl_data.head(10))
data_with_imputed_values = my_imputer.fit_transform(nfl_data)
print(data_with_imputed_values.head(10))

ModuleNotFoundError: No module named 'sklearn.impute'

In [ ]:
import sklearn
print (sklearn.__version__)
!pip install git+https://github.com/scikit-learn/scikit-learn.git

0.19.1
